In [31]:
import pandas as pd
import seaborn as sb

import random
import math

In [32]:
class DataProcessing:
    @staticmethod
    def shuffling(data_list):
        for i in range(len(data_list)-1,0,-1):
            index = random.randint(0,i-1)
            data_list.loc[i], data_list.loc[index] = data_list.loc[index], data_list.loc[i]

    @staticmethod
    def normalize(data_list):
        for col in data_list.columns:
            if type(data_list[col].loc[0]) is not type("text"):
                min1 = float("inf")
                max1 = float("-inf")

                for x in data_list[col]:
                    min1 = min(min1,x)
                    max1 = max(max1,x)
                    
                for i in range(len(data_list[col])):
                    data_list.at[i, col] -= min1  
                    data_list.at[i, col] /= (max1 - min1)

    @staticmethod
    def train_test_split(data_list):
        train_len = round((len(data_list)) * 0.7)
        #test_len = round((len(data_list)) * 0.3)
        
        train = data_list[0:train_len]
        test = data_list[train_len:len(data_list)]

        test = test.reset_index(drop=True) #resetowanie indeksacji dataframeu

        return train, test
    
    @staticmethod
    def label_split(data_list):
        feature_list = []
        label_list = []

        sum = 0
        for col in data_list.columns:
            if type(data_list[col].loc[0]) is not type("text"):
                sum +=1
        
        for i in range(len(data_list)):
            feature_list.append(data_list.loc[i].to_list()[:sum])

        for i in range(len(data_list)):
            label_list.append(data_list.loc[i].to_list()[sum:])
        
        return feature_list, label_list

In [33]:
data = pd.read_csv("iris.csv")

#print(len(data)) # dlugosc datasetu
#print(data.loc[5]) # uzyskiwanie danych o jednym rekordzie
#print(data.loc[5].to_list()) # wypisywanie rekordu w postaci listy

#sb.pairplot(data, hue="variety") # wyswietlanie wykresow kazdej cechy
#sb.violinplot(data, x="sepal.width", y="variety", inner="quartile")
#print(data.describe()) # wypisuje uśrednione info o datasecie
#print(data)

DataProcessing.shuffling(data)
DataProcessing.normalize(data)
train_data, test_data = DataProcessing.train_test_split(data)

#print(train_data)
#print(test_data)



     sepal.length  sepal.width  petal.length  petal.width     variety
0        0.027778     0.416667      0.050847     0.041667      Setosa
1        0.305556     0.791667      0.118644     0.125000      Setosa
2        0.361111     0.291667      0.542373     0.500000  Versicolor
3        0.194444     0.125000      0.389831     0.375000  Versicolor
4        0.611111     0.416667      0.813559     0.875000   Virginica
..            ...          ...           ...          ...         ...
100      0.666667     0.416667      0.677966     0.666667  Versicolor
101      0.527778     0.333333      0.644068     0.708333   Virginica
102      0.527778     0.583333      0.745763     0.916667   Virginica
103      0.361111     0.416667      0.593220     0.583333  Versicolor
104      0.361111     0.416667      0.525424     0.500000  Versicolor

[105 rows x 5 columns]
    sepal.length  sepal.width  petal.length  petal.width     variety
0       0.500000     0.375000      0.627119     0.541667  Versicolo